In [9]:
import pandas as pd
from kiwipiepy import Kiwi
import re
import copy


# 필요한 객체 및 변수 정의
kiwi = Kiwi(typos='basic')

In [10]:
query = '기독교' # 분석 대상 키워드
df = pd.read_csv(f'./news_pre/Selenium_news_{query}.csv') #(뉴스)
df = df.dropna(subset=['content'],ignore_index=True) # 본문(content) 스크래핑 결과가 Nan인 값을 제거

In [11]:
# 각 종교별 키워드를 리스트에 담아서 불용어 처리에 사용 (스크래핑 검색 단어가 많이 추출되는 거는 당연해서 분석의미가 없음)
# 블로그 글 특성상 자주 나오지만 분석에 도움되지 않는 단어도 리스트로 담아 저장 (모델 수행 결과를 바탕으로 경험적으로 추가)
gidok_stop = ['기독교','교회','하나님','개신교','목사','예수']
chunju_stop = ['천주교','수녀님','성당','하느님','신부님','교황']
bul_stop = ['불교','부처님','절','스님','승려','보살']
blog_stop = ['이웃','본문','추가','복사','기타','분','곳','번']

In [12]:
from kiwipiepy.utils import Stopwords # kiwi에서 제공하는 Stopword list
stopwords = Stopwords()

# 기본 불용어 리스트 로드


# 기본 불용어 + 사용자 불용어 통합
default_stopwords = stopwords.stopwords
stopwords = set(default_stopwords)  # 기본 불용어를 set으로 변환
custom_stopwords = set(gidok_stop + chunju_stop + bul_stop + blog_stop)

stopwords.update((word, 'NNG') for word in custom_stopwords)  # 사용자 불용어 추가 (보통명사)
stopwords.update((word, 'NNP') for word in custom_stopwords)  # 사용자 불용어 추가 (고유명사)

In [13]:
stopwords

{('ᆫ', 'ETM'),
 ('ᆫ', 'JX'),
 ('ᆫ다', 'EF'),
 ('ᆯ', 'ETM'),
 ('가', 'JKS'),
 ('같', 'VA'),
 ('개신교', 'NNG'),
 ('개신교', 'NNP'),
 ('것', 'NNB'),
 ('게', 'EC'),
 ('겠', 'EP'),
 ('고', 'EC'),
 ('고', 'JKQ'),
 ('곳', 'NNG'),
 ('곳', 'NNP'),
 ('과', 'JC'),
 ('과', 'JKB'),
 ('교황', 'NNG'),
 ('교황', 'NNP'),
 ('교회', 'NNG'),
 ('교회', 'NNP'),
 ('그', 'MM'),
 ('그', 'NP'),
 ('기', 'ETN'),
 ('기독교', 'NNG'),
 ('기독교', 'NNP'),
 ('기타', 'NNG'),
 ('기타', 'NNP'),
 ('까지', 'JX'),
 ('나', 'NP'),
 ('년', 'NNB'),
 ('는', 'ETM'),
 ('는', 'JX'),
 ('다', 'EC'),
 ('다', 'EF'),
 ('다고', 'EC'),
 ('다는', 'ETM'),
 ('대하', 'VV'),
 ('더', 'MAG'),
 ('던', 'ETM'),
 ('도', 'JX'),
 ('되', 'VV'),
 ('되', 'XSV'),
 ('들', 'XSN'),
 ('등', 'NNB'),
 ('따르', 'VV'),
 ('때', 'NNG'),
 ('때문', 'NNB'),
 ('라', 'EC'),
 ('라는', 'ETM'),
 ('로', 'JKB'),
 ('를', 'JKO'),
 ('만', 'JX'),
 ('만', 'NR'),
 ('말', 'NNG'),
 ('며', 'EC'),
 ('면', 'EC'),
 ('면서', 'EC'),
 ('명', 'NNB'),
 ('목사', 'NNG'),
 ('목사', 'NNP'),
 ('받', 'VV'),
 ('번', 'NNG'),
 ('번', 'NNP'),
 ('보', 'VV'),
 ('보살', 'NNG'),
 ('보살', 'NN

In [14]:
# 기본 불용어 + 사용자 불용어 통합
stopwords = set((word, 'NNG') for word in default_stopwords)  # (단어, 품사) 형태로 변환
stopwords.update((word, 'NNG') for word in custom_stopwords)  # 사용자 불용어 추가
stopwords.update((word, 'NNP') for word in custom_stopwords)  # 고유명사(NNP)도 추가

In [15]:
def preprocess_korean(text, analyzer=kiwi, stopwords=stopwords, return_list=False):
    my_text = str(copy.copy(text)).replace('\n', ' ')  # 개행 제거
    my_text = analyzer.space(my_text)  # 띄어쓰기 교정

    # Tokenizer
    sents = analyzer.split_into_sents(my_text)
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')  # 한글만 남기는 정규식
    all_result = []

    for sent in sents:
        token_result = analyzer.tokenize(sent.text)
        tmp = []  # 
        for token in token_result:
            # 명사(NNG, NNP)만 추출 & 불용어 제거
            if (token.form, token.tag) not in stopwords and token.tag in ['NNG', 'NNP']:
                tmp.append(token.form)

        token_result = ' '.join(tmp)
        token_result = p.sub(' ', token_result)  # 한글 이외 문자 제거
        all_result.append(token_result)

    return all_result if return_list else ' '.join(all_result)  # 리스트로 반환 옵션 추가


In [19]:
df1 = df.head(1)

In [20]:
df1['pre_content'] = df1['content'].apply(lambda x: preprocess_korean(x)) # 전처리기 적용

C:\Users\wngud\AppData\Local\Temp\ipykernel_1396\2455809945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['pre_content'] = df1['content'].apply(lambda x: preprocess_korean(x)) # 전처리기 적용


In [ ]:
#df['pre_content'] = df['content'].apply(lambda x: preprocess_korean(x)) # 전처리기 적용

In [21]:
a = df1['content'][0]
b = df1['pre_content'][0]

In [22]:
a

'밀양농아인교회 내달 30일까지2024 십자가 전시회판매 수익금 장애인 선교에 사용밀양농아인교회(천형기 목사)는 지난 14일부터 경남 밀양시 단장면 상봉1길 70 교회에서 ‘2024 십자가 전시회’를 개최하고 있다. (사진)지난해에 이어 열리는 이 전시회의 주제는 ‘그리스도의 십자가’(갈 6:14)이다.농인 최호식 목사, 장기현 집사 등 작가 10여명이 제작한 다양한 십자가와 성서 유물을 전시한다.3개의 대형 십자가 묵상, 십자가 자기 체험을 하고 기념품을 증정한다.전시회는 다음 달 30일까지다. 관람시간은 평일 토요일 오전 10시~오후 5시, 주일 오후 2~5시이다.천형기 밀양농아인교회 목사는 초청의 글에서 “십자가와 부활은 기독교 신앙에서 가장 중요한 핵심”이라며 “그러나 점점 십자가에 대한 의미를 잃어버리고 살 때가 많다. 사순절 기간 예수 십자가의 고난과 의미를 되새기며 다시 십자가를 새롭게 바라보길 바란다”고 밝혔다.천 목사는 “어려운 조건에서 십자가를 만드는 농인들을 계속 알리고 조금이나마 도움이 되었으면 하는 바람”이라고 말했다.행사는 밀양시민교회(나철수 목사), 마산봉덕교회(강정식 목사) 등이 협력하고 있다.판매 수익금은 장애인 선교를 위해 사용한다.'

In [23]:
b

'밀양 농아 내달 십자가 전시회 판매 수익금 장애인 선교 사용 밀양 농아 천 형기 경남 밀양시 단장면 상봉 길 십자가 전시회 개최 사진 지난해 전시회 주제 그리스도 십자가 갈 농 최호식 장기현 집사 작가 제작 다양 십자가 성서 유물 전시 대형 십자가 묵상 십자가 자기 체험 기념품 증정 전시회 다음 달 관람 시간 평일 토요일 오전 오후 주일 오후 천 형기 밀양 농아 초청 글 십자가 부활 신앙 중요 핵심 십자가 의미 때 사순절 기간 십자가 고난 의미 십자가 천 조건 십자가 농 조금 도움 바람 말 행사 밀양 시민 나 철수 마산봉덕교회 강 정식 협력 판매 수익금 장애인 선교 사용'

In [24]:
df.to_csv(f'../wordcloud/preprocessed_{query}.csv', index=False)  # 전처리된 결과를 담은 csv 저장